In [ ]:
Ibarbecue refers to the segment of Texan cuisine that specializes in grilled and smoked meats, as well as the many side dishes that traditionntroduction
Texas ally accompany them. Beef brisket and various pork products are considered the state specialty. In some instances, Texas-style barbecue has merged with another state mainstay, Tex-Mex cuisine. It's quite plausible to find a barbecue restaurant that offers tacos alongside spare ribs and brisket sandwiches. 
The central European traditions of these immigrants placed a heavy emphasis on meats and sausages. These were sold fresh in markets, but the merchants were aware that meat could be preserved far longer if it was smoked. They began smoking and slow cooking meats that did not promptly sell, and the rich, smoky flavor of these meats became desirable in and of itself.  Over time, the popularity of barbecued meat spread throughout the state, and while central Texas remains its epicenter, you can easily find BBQ joints from Tyler in the east to El Paso in the west, to say nothing of big cities like Dallas, Fort Worth, Houston, Austin, and San Antonio.

Because Texas is so popular for BBQ restaurants, finding an available corner or street that does not have a barbecue restaurant is challenging.  The problem we aim to solve is ‘Which city is best for opening a barbecue restaurant in North Texas?’ The main target is North Texas locals with minimum choices for BBQ restaurants

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 112kB 8.6MB/s eta 0:00:01
Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'JBIECKAD2LZMNKSNDLZYCJ5HWK0S3RLLZRYSHTQB0RSV2SOH' # your Foursquare ID
CLIENT_SECRET = 'CHFHYLNUHHO1MLEPUUMFDLRDYPBGFOBROALUTZDOWLDGQRND' # your Foursquare Secret
ACCESS_TOKEN = '4WA54JBRX0ZTKJ2TURKI4EG4IT4FSNJDSWG2IS5N5002E2JF' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JBIECKAD2LZMNKSNDLZYCJ5HWK0S3RLLZRYSHTQB0RSV2SOH
CLIENT_SECRET:CHFHYLNUHHO1MLEPUUMFDLRDYPBGFOBROALUTZDOWLDGQRND


In [7]:
# type your answer here
LIMIT = 100 # Maximum is 100
cities = ["Fort Worth, TX", 'Arlington, TX', 'Burleson, TX', 'Dallas, TX']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            city,
            LIMIT,
            "4bf58dd8d48988d1df931735") # BBQ CATEGORY ID
    results[city] = requests.get(url).json()

Define Functions

In [8]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [9]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of BBQ places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of BBQ places in Fort Worth, TX =  122
Showing Top 100
Total number of BBQ places in Arlington, TX =  72
Showing Top 100
Total number of BBQ places in Burleson, TX =  57
Showing Top 100
Total number of BBQ places in Dallas, TX =  232
Showing Top 100


In [11]:
 print("Fort Worth, Texas")
maps[cities[0]]

Fort Worth, Texas


In [12]:
print("Arlington, Texas")
maps[cities[1]]

Arlington, Texas


In [13]:
print("Burleson, Texas")
maps[cities[2]]

Burleson, Texas


In [14]:
print("Dallas, Texas")
maps[cities[3]]

Dallas, Texas


We can see that there is a heavy concentration of BBQ resturants in Fort Worth and Dallas in North Texas. 

However, Let's have a concrete measure of this density.

For this I will use some basic statistics. I will get the mean location of the BBQ places which should be near to most of them if they are really dense or far if not.
Next I will take the average of the distance of the venues to the mean coordinates.

In [108]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Fort Worth, TX
Mean Distance from Mean coordinates
0.08409948269797886
Arlington, TX
Mean Distance from Mean coordinates
0.046168187171347894
Burleson, TX
Mean Distance from Mean coordinates
0.33792457447344615
Dallas, TX
Mean Distance from Mean coordinates
0.11216963605687466


In [109]:
maps[cities[0]]

In [110]:
maps[cities[1]]

In [111]:
maps[cities[2]]

In [112]:
maps[cities[3]]

We now see that Burleson is the best city to start a BBQ business.  Entrepurantn would be more successful and incur less risk of failure in Burleson.

Enjoy that BBQ!